In [1]:
import sys
from itertools import count
from torch import autograd
from torch_geometric.utils import dense_to_sparse
import copy
from collections import defaultdict

sys.path.append('../../')

from src.models.gcn import *
from src.utils.datasets import *
from src.models.trainable import *
from src.attacks.greedy_gd import *

print(sys.executable)

/home/niyati/miniconda3/envs/ersp_v2/bin/python


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
# dataset_directory = "../Cora"
citeseer_dataset = Planetoid(root='', name='Citeseer')
data = citeseer_dataset[0].to(device)
print(data)

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [4]:
model = GCN(data.x.shape[1], citeseer_dataset.num_classes, [64]).to(device)

In [5]:
model.reset_parameters()
train = Trainable(model)
train.fit(data, 200)

Epoch 0, Train Loss - 3.7768473625183105, Val Loss - 2.87339448928833, Val Accuracy - 0.414
Epoch 20, Train Loss - 0.12260017544031143, Val Loss - 4.737508773803711, Val Accuracy - 0.548
Epoch 40, Train Loss - 0.012804927304387093, Val Loss - 5.134322166442871, Val Accuracy - 0.578
Epoch 60, Train Loss - 0.01174915675073862, Val Loss - 5.340953350067139, Val Accuracy - 0.564
Epoch 80, Train Loss - 0.005575299728661776, Val Loss - 4.999581336975098, Val Accuracy - 0.604
Epoch 100, Train Loss - 0.0041719707660377026, Val Loss - 5.084325313568115, Val Accuracy - 0.582
Epoch 120, Train Loss - 0.003931032959371805, Val Loss - 5.00808572769165, Val Accuracy - 0.588
Epoch 140, Train Loss - 0.011622863821685314, Val Loss - 4.810202598571777, Val Accuracy - 0.593
Epoch 160, Train Loss - 0.0018068465869873762, Val Loss - 4.755350589752197, Val Accuracy - 0.606
Epoch 180, Train Loss - 0.0029572665225714445, Val Loss - 5.887982368469238, Val Accuracy - 0.551
Epoch 200, Train Loss - 0.0039050329942

In [6]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.645
Initial Loss: 5.194754600524902


In [8]:
amts = defaultdict(int)
# Run 10 attacks for perturbation of 0.2
for i in range(10):
    attacker = Metattack(data, device=device)
    attacker.setup_surrogate(model,
                         labeled_nodes=data.train_mask,
                         unlabeled_nodes=data.test_mask, lambda_=0.)
    attacker.reset_parameters(seed=i)
    attacker.reset()
    attacker.attack(0.1)

    for edge in attacker._added_edges.keys():
        amts[edge] += 1


Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

Peturbing graph...:   0%|          | 0/455 [00:00<?, ?it/s]

In [9]:
amts

defaultdict(int,
            {(91, 105): 10,
             (91, 1815): 10,
             (113, 3177): 10,
             (43, 3320): 10,
             (107, 3236): 10,
             (99, 3058): 10,
             (43, 2915): 10,
             (119, 3091): 10,
             (89, 91): 10,
             (90, 2231): 10,
             (91, 1674): 10,
             (91, 3144): 10,
             (26, 1090): 10,
             (91, 94): 10,
             (91, 1523): 10,
             (91, 973): 10,
             (8, 91): 10,
             (6, 91): 10,
             (91, 855): 10,
             (91, 1514): 10,
             (52, 2766): 10,
             (91, 2013): 10,
             (48, 1128): 10,
             (91, 776): 10,
             (91, 2508): 10,
             (43, 2568): 10,
             (53, 1096): 10,
             (39, 91): 10,
             (38, 105): 10,
             (91, 1900): 10,
             (91, 2392): 10,
             (113, 3001): 10,
             (91, 326): 10,
             (5, 2794): 10,
            

In [10]:
sorted_list = sorted(amts.items(), key=lambda item: item[1], reverse=True)
sorted_keys = [key for key, value in sorted_list]

In [11]:
edges_to_add = sorted_keys

In [12]:
from torch_geometric.utils import dense_to_sparse, to_networkx, from_networkx
import networkx as nx

In [13]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.19
budget = int(ptb_rate * initial_edge_count)

In [14]:
print(len(edges_to_add))
print(budget)

446
432


In [15]:
def two_phase_attack(split):
    diff_threshold = 0.01
    first_phase_edges = int(budget * split)
    second_phase_percent = ptb_rate * (1 - split) * 0.5
    print(f"\n--- Running split: {split} ---")
    print(f"Second phase perturbation rate: {second_phase_percent:.4f}")

    phase1_accuracies = []
    phase2_accuracies = []

    G = to_networkx(data, to_undirected=True)
    data_copy = copy.copy(data)

    i, j = 0, 0  # i - edges successfully added, j - index in list

    # === Phase 1 ===
    while i < first_phase_edges:
        if j >= len(edges_to_add):
            print("Ran out of candidate edges in Phase 1. Moving to Phase 2.")
            break

        u, v = edges_to_add[j]
        G.add_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)

        if modified_accuracy == initial_accuracy:
            i += 1
            phase1_accuracies.append(modified_accuracy)
        else:
            G.remove_edge(u, v)

        j += 1

    print(f"Phase 1: Added {i} edges out of requested {first_phase_edges}.")

    # === Phase 2 ===
    modified_data = from_networkx(G).to(device)
    modified_data.x = data.x 
    modified_data.y = data.y 
    modified_data.train_mask = data.train_mask
    modified_data.test_mask = data.test_mask

    attacker = Metattack(modified_data, device=device)
    attacker.setup_surrogate(model,
                             labeled_nodes=data.train_mask,
                             unlabeled_nodes=data.test_mask, 
                             lambda_=0.)
    attacker.reset()
    attacker.attack(second_phase_percent)

    degs = defaultdict(tuple)
    for k, v in attacker._added_edges.items():
        degs[v] = (k, True)
    for k, v in attacker._removed_edges.items():
        degs[v] = (k, False)

    for _, second in degs.items():
        u, v = second[0]
        if second[1]:
            G.add_edge(u, v)
        else:
            G.remove_edge(u, v)

        modified_data = from_networkx(G).to(device)
        modified_data.x = data.x 
        modified_data.y = data.y 
        modified_data.train_mask = data.train_mask
        modified_data.test_mask = data.test_mask

        _, modified_accuracy = train.test(modified_data)
        phase2_accuracies.append(modified_accuracy)

    # === Final Reporting ===
    final_accuracy = phase2_accuracies[-1] if phase2_accuracies else (
        phase1_accuracies[-1] if phase1_accuracies else initial_accuracy)
    accuracy_drop = initial_accuracy - final_accuracy

    print(f"Final Accuracy: {final_accuracy:.4f}")
    print(f"Accuracy Drop: {accuracy_drop:.4f}")

    return {
        "split": split,
        "phase1_added": i,
        "phase1_accuracies": phase1_accuracies,
        "phase2_accuracies": phase2_accuracies,
        "final_accuracy": final_accuracy,
        "accuracy_drop": accuracy_drop
    }


In [16]:
splits = [0, 0.5]
split_dic = defaultdict(list)

In [17]:
for s in splits:
    print(s)
    split_dic[s] = two_phase_attack(s)

0

--- Running split: 0 ---
Second phase perturbation rate: 0.0950
Phase 1: Added 0 edges out of requested 0.


Peturbing graph...:   0%|          | 0/432 [00:00<?, ?it/s]

Final Accuracy: 0.6340
Accuracy Drop: 0.0110
0.5

--- Running split: 0.5 ---
Second phase perturbation rate: 0.0475
Phase 1: Added 216 edges out of requested 216.


Peturbing graph...:   0%|          | 0/226 [00:00<?, ?it/s]

Final Accuracy: 0.6230
Accuracy Drop: 0.0220


In [19]:
G = to_networkx(data, to_undirected=True)
initial_edge_count = G.number_of_edges() // 2
ptb_rate = 0.5
budget = int(ptb_rate * initial_edge_count)

In [20]:
print(len(edges_to_add))
print(budget)

724
1319
